# Coastal protection widget  
Data exploration and preparation for the coastal protection data.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import product

Load data from bucket

In [2]:
df = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/Mangrove_Coastal_Protection_Stats_20230426.csv', skiprows=1)
df.head()

,Name,Area,Population,Stock,Area.1,Population.1,Stock.1,Area.2,Population.2,Stock.2
0,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Angola,NaN,3128.645656,2.381327e+06,0.000000,7241.891811,3.939056e+06,0.000000,14039.010650,12073501.76
2,Anguilla,0.615599,0.000000,5.042979e+06,0.922914,0.000000,2.926016e+06,2.781378,0.000000,7497182.30
3,Antigua and Barbuda,2.375643,312.024456,1.553655e+07,2.835863,1208.306314,3.053302e+07,9.206714,1144.118241,70476111.40
4,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Generate standard names for the columns

In [3]:
period = ['annual', '25_year', '100_year']
impact = ['area', 'population', 'stock']
cols_names = []

for p, i in product(period, impact):
    cols_names.append(f'{p}_{i}')
cols_names

['annual_area',
 'annual_population',
 'annual_stock',
 '25_year_area',
 '25_year_population',
 '25_year_stock',
 '100_year_area',
 '100_year_population',
 '100_year_stock']

In [4]:
df.columns = ['country'] + cols_names
df.head(1)

,country,annual_area,annual_population,annual_stock,25_year_area,25_year_population,25_year_stock,100_year_area,100_year_population,100_year_stock
0,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Check number of NAs per row, eliminate rows with all NAs (sum = 9)

In [5]:
df['number_na'] = df.isna().sum(axis=1)
df['number_na'].value_counts()

0    95
9    27
1     1
Name: number_na, dtype: int64

In [6]:
df = df[df['number_na'] < 9]
print(f'Rows with data: {len(df)}')

Rows with data: 96


In [7]:
df_long = pd.melt(df, id_vars=['country'], value_vars=cols_names)
df_long.rename(columns={'variable':'indicator'}, inplace=True)
df_long['period'] = df_long['indicator'].str.split('_').str[0].replace({'25':'25_year', '100':'100_year'})
df_long['indicator'] = df_long['indicator'].str.split('_').str[-1]
df_long

,country,indicator,value,period
0,Angola,area,NaN,annual
1,Anguilla,area,6.155990e-01,annual
2,Antigua and Barbuda,area,2.375643e+00,annual
3,Australia,area,5.879710e+01,annual
4,Bahamas,area,2.470507e+03,annual
...,...,...,...,...
859,Vietnam,stock,7.473397e+09,100_year
860,British Virgin Islands,stock,2.444817e+07,100_year
861,"Virgin Islands, U.S.",stock,1.503067e+07,100_year
862,Yemen,stock,1.357952e+07,100_year


In [8]:
units_df = pd.DataFrame({'indicator': impact, 'unit': ['km2', 'people', 'USD']})
units_df

,indicator,unit
0,area,km2
1,population,people
2,stock,USD


In [9]:
df_long = df_long.merge(units_df, on='indicator')
df_long

,country,indicator,value,period,unit
0,Angola,area,NaN,annual,km2
1,Anguilla,area,6.155990e-01,annual,km2
2,Antigua and Barbuda,area,2.375643e+00,annual,km2
3,Australia,area,5.879710e+01,annual,km2
4,Bahamas,area,2.470507e+03,annual,km2
...,...,...,...,...,...
859,Vietnam,stock,7.473397e+09,100_year,USD
860,British Virgin Islands,stock,2.444817e+07,100_year,USD
861,"Virgin Islands, U.S.",stock,1.503067e+07,100_year,USD
862,Yemen,stock,1.357952e+07,100_year,USD


In [10]:
df_long.to_csv('../../../../data/coastal_protection_tableau.csv', index=False)